In [27]:
from gvol import GVol
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_rows', 500)
#pd.reset_option('display.float_format')

import numpy as np
import mibian

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import datetime
from datetime import date

import scipy.interpolate as interpolate

import ccxt
deribit = ccxt.deribit()

import warnings
warnings.filterwarnings('ignore')

pd.options.plotting.backend = "plotly"

# gvol_client = GVol(header='x-oracle',gvol_api_key="GVOL_API_KEY")
###LITE
gvol_client = GVol(header='gvol-lite',gvol_api_key="34de596a-732e-4199-9d2a-4348b687f1fa")


from colour import *

colore = px.colors.sequential.thermal
if len(colore[0])<17:
    colore.reverse()

###BSM formula for greeks calculation
riskfree_rate = 0

def BSM(x):
    return (mibian.BS([x['indexPrice'],x['strike'],riskfree_rate,x['DTE']], volatility = x['markIv']))

#  fig.write_html("path/to/file.html")

# VOLATILITY CHARTS FOR SELECTED EXCHANGE

In [28]:
##SELECT EXCHANGE: (Supported exchanges are |deribit|bitcom|okex|delta|)
exchange = 'deribit'

In [29]:
data = gvol_client.options_orderbook_details(exchange=exchange)
data = pd.json_normalize(data['UtilityRealtimeOptionbook'])
data['date'] = pd.to_datetime(data['date'],unit='ms')
data['expiration'] = pd.to_datetime(data['expiration'],unit='ms')

data['DTE'] = pd.to_datetime(data['expiration'])- pd.to_datetime(data['date'],unit='ms')
data['DTE'] = data['DTE'].apply(datetime.timedelta.total_seconds) / (60*60*24)

data['BSM'] = data.apply(BSM,axis=1)
data['delta_BSM'] = data.apply(lambda x: x['BSM'].callDelta if x['putCall']=='C' else x['BSM'].putDelta, axis=1)
data['price_BSM'] = data.apply(lambda x: x['BSM'].callPrice if x['putCall']=='C' else x['BSM'].putPrice, axis=1)
data.loc[data['delta_BSM']>0,'delta_grid'] = 1 - data.loc[data['delta_BSM']>0]['delta_BSM']
data.loc[data['delta_BSM']<0,'delta_grid'] = -1 * data.loc[data['delta_BSM']<0]['delta_BSM']

#BUILD MONEYNESS
data.loc[(data['delta_grid']<0.50)&(data['putCall']=='P'),'moneyness'] = 'OTM'
data.loc[(data['delta_grid']>=0.50)&(data['putCall']=='P'),'moneyness'] = 'ITM'
data.loc[(data['delta_grid']<0.50)&(data['putCall']=='C'),'moneyness'] = 'ITM'
data.loc[(data['delta_grid']>=0.50)&(data['putCall']=='C'),'moneyness'] = 'OTM'

In [30]:
data.columns

Index(['date', 'instrumentName', 'currency', 'expiration', 'strike', 'putCall',
       'isAtm', 'oi', 'bestBidPrice', 'bestAskPrice', 'usdBid', 'usdAsk',
       'bidIV', 'markIv', 'askIv', 'indexPrice', 'underlyingPrice', 'DTE',
       'BSM', 'delta_BSM', 'delta_grid', 'moneyness'],
      dtype='object')

In [32]:
data

,date,instrumentName,currency,expiration,strike,putCall,isAtm,oi,bestBidPrice,bestAskPrice,...,bidIV,markIv,askIv,indexPrice,underlyingPrice,DTE,BSM,delta_BSM,delta_grid,moneyness
0,2022-10-14 13:46:08.218,BTC-15OCT22-22000-C,BTC,2022-10-15 08:00:00,22000,C,False,29.9000,0.0000,0.0005,...,0.0000,88.4700,123.5600,19647.0400,19649.1936,0.7596,<mibian.BS object at 0x000001F7D1D4B850>,0.0027,0.9973,OTM
1,2022-10-14 13:46:08.218,BTC-15OCT22-22000-P,BTC,2022-10-15 08:00:00,22000,P,False,0.0000,0.0000,1.0000,...,0.0000,88.4700,500.0000,19647.0400,19649.1936,0.7596,<mibian.BS object at 0x000001F7D1D4B5E0>,-0.9973,0.9973,ITM
2,2022-10-14 13:46:10.229,BTC-15OCT22-21000-C,BTC,2022-10-15 08:00:00,21000,C,False,165.1000,0.0000,0.0005,...,0.0000,67.4800,80.0900,19642.7700,19641.8567,0.7596,<mibian.BS object at 0x000001F7D1D4B370>,0.0156,0.9844,OTM
3,2022-10-14 13:46:08.211,BTC-15OCT22-21000-P,BTC,2022-10-15 08:00:00,21000,P,False,0.0000,0.0000,1.0000,...,0.0000,67.4800,500.0000,19647.0400,19649.1936,0.7596,<mibian.BS object at 0x000001F7D1D4B1F0>,-0.9841,0.9841,ITM
4,2022-10-14 13:46:08.212,BTC-15OCT22-20500-C,BTC,2022-10-15 08:00:00,20500,C,False,112.8000,0.0005,0.0010,...,55.4200,57.0300,64.3900,19647.0400,19649.1936,0.7596,<mibian.BS object at 0x000001F7D1D4B130>,0.0526,0.9474,OTM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,2022-10-14 13:45:41.196,SOL-30DEC22-20-P,SOL,2022-12-30 08:00:00,20,P,False,3072.0000,0.0310,0.0340,...,99.8700,101.5900,103.2400,31.6054,31.3620,76.7599,<mibian.BS object at 0x000001F7D0962970>,-0.1122,0.1122,OTM
1292,2022-10-14 13:46:10.414,SOL-30DEC22-15-C,SOL,2022-12-30 08:00:00,15,C,False,0.0000,0.0000,0.0000,...,0.0000,118.1800,0.0000,31.5806,31.3492,76.7596,<mibian.BS object at 0x000001F7D09629A0>,0.9500,0.0500,ITM
1293,2022-10-14 13:46:10.412,SOL-30DEC22-15-P,SOL,2022-12-30 08:00:00,15,P,False,2655.0000,0.0130,0.0160,...,114.7500,118.1800,120.9900,31.5806,31.3507,76.7596,<mibian.BS object at 0x000001F7D09629D0>,-0.0500,0.0500,OTM
1294,2022-10-14 13:46:10.412,SOL-30DEC22-10-C,SOL,2022-12-30 08:00:00,10,C,False,0.0000,0.0000,0.0000,...,0.0000,143.2300,0.0000,31.5806,31.3507,76.7596,<mibian.BS object at 0x000001F7D0962A00>,0.9812,0.0188,ITM


In [3]:
data = gvol_client.options_orderbook_details(exchange=exchange)
data = pd.json_normalize(data['UtilityRealtimeOptionbook'])
data['date'] = pd.to_datetime(data['date'],unit='ms')
data['expiration'] = pd.to_datetime(data['expiration'],unit='ms')

data['DTE'] = pd.to_datetime(data['expiration'])- pd.to_datetime(data['date'],unit='ms')
data['DTE'] = data['DTE'].apply(datetime.timedelta.total_seconds) / (60*60*24)

data['BSM'] = data.apply(BSM,axis=1)
data['delta_BSM'] = data.apply(lambda x: x['BSM'].callDelta if x['putCall']=='C' else x['BSM'].putDelta, axis=1)
data.loc[data['delta_BSM']>0,'delta_grid'] = 1 - data.loc[data['delta_BSM']>0]['delta_BSM']
data.loc[data['delta_BSM']<0,'delta_grid'] = -1 * data.loc[data['delta_BSM']<0]['delta_BSM']

#BUILD MONEYNESS
data.loc[(data['delta_grid']<0.50)&(data['putCall']=='P'),'moneyness'] = 'OTM'
data.loc[(data['delta_grid']>=0.50)&(data['putCall']=='P'),'moneyness'] = 'ITM'
data.loc[(data['delta_grid']<0.50)&(data['putCall']=='C'),'moneyness'] = 'ITM'
data.loc[(data['delta_grid']>=0.50)&(data['putCall']=='C'),'moneyness'] = 'OTM'

###FILTERING INSTRUMENTS FOR HAVING BETTER FITTING ON CURVE
data = data[(data['delta_grid'].between(0.02,0.97,inclusive=True))]

data = data[data['moneyness']=='OTM']

In [9]:
data.columns

Index(['date', 'instrumentName', 'currency', 'expiration', 'strike', 'putCall',
       'isAtm', 'oi', 'bestBidPrice', 'bestAskPrice', 'usdBid', 'usdAsk',
       'bidIV', 'markIv', 'askIv', 'indexPrice', 'underlyingPrice', 'DTE',
       'BSM', 'delta_BSM', 'delta_grid', 'moneyness'],
      dtype='object')

In [4]:
currencies = data.currency.unique()
currencies

array(['BTC', 'ETH', 'SOL'], dtype=object)

In [5]:
###Select the pair of currencies to be compared in delta grid chart
curr1 = currencies[0]
curr2 = currencies[-1]

In [6]:
#DEFINE SUBPLOTS CHART
fig = make_subplots(rows=1,cols=2,subplot_titles=(curr1, curr2,))

##BTC DELTA GRID CHART
orderbookskew = data[(data['currency']==curr1)&(data['moneyness']=='OTM')]
i = 0
for e in orderbookskew['expiration'].unique():
    
    orderbookskew_exp = orderbookskew[orderbookskew['expiration']==e]
    orderbookskew_exp = orderbookskew_exp.sort_values('delta_grid')
  
    if len(orderbookskew_exp)<=3:
        continue

    delta_grid = orderbookskew_exp['delta_grid'].values
    markIv = orderbookskew_exp['markIv'].values
    
    
    volfit_param = interpolate.splrep(delta_grid, markIv, s=4)
        
    strikes_random = np.linspace(0.05,0.95,100)
    
    x = strikes_random
    y = interpolate.splev(strikes_random, volfit_param,ext=3)
      
    
    fig.add_trace(go.Scatter(x=x, y=y,
                  name=curr1+" "+str(e)[2:10],
                  line= dict(color=colore[i])),
                  row=1, col=1
                 )
    
    
    i = i + 1


##ETH DELTA GRID CHART
orderbookskew = data[(data['currency']==curr2)&(data['moneyness']=='OTM')]
i = 0
for e in orderbookskew['expiration'].unique():
    
    orderbookskew_exp = orderbookskew[orderbookskew['expiration']==e]
    orderbookskew_exp = orderbookskew_exp.sort_values('delta_grid')
    
    if len(orderbookskew_exp)<=3:
        continue
  
    delta_grid = orderbookskew_exp['delta_grid'].values
    markIv = orderbookskew_exp['markIv'].values
    
    
    volfit_param = interpolate.splrep(delta_grid, markIv, s=4)
        
    strikes_random = np.linspace(0.05,0.95,100)
    
    x = strikes_random
    y = interpolate.splev(strikes_random, volfit_param,ext=3)
      
    
    fig.add_trace(go.Scatter(x=x, y=y,
                  name=curr2+" "+str(e)[2:10],
                  line= dict(color=colore[i])),
                  row=1, col=2
                 )
    
    
    i = i + 1
    
#GLOBAL CHARTS PARAMETERS
fig.update_layout(showlegend=True)
fig.update_xaxes(range=[0.05,0.95])
#base = max(50,min(y)-5)
fig.update_yaxes(range=[50,120])

fig.update_layout(title=exchange.upper()+" VOL CURVE DELTA GRID",title_x=0.50,
                  yaxis_title="volatility",
                  legend=dict(
                      orientation="h",
                      yanchor="bottom",
                      y=-0.25,
                      xanchor="right",
                      x=1.01
                      ),
                   #['ggplot2', 'seaborn', 'simple_white', 'plotly',
                   #'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
                   #'ygridoff', 'gridon', 'none']
                  template = 'gridon')
fig

In [10]:
#nr_curr = len(data.currency.unique())
#DEFINE SUBPLOTS CHART


for curr in data.currency.unique():
    
    fig = make_subplots(rows=1,cols=1)

    ##BTC DELTA GRID CHART
    orderbookskew = data[data['currency']==curr]
    i = 0
    for e in orderbookskew['expiration'].unique():

        orderbookskew_exp = orderbookskew[orderbookskew['expiration']==e]
        orderbookskew_exp = orderbookskew_exp.sort_values('strike')
        
        if len(orderbookskew_exp)<=3:
            continue
            
        strike = orderbookskew_exp['strike'].values
        markIv = orderbookskew_exp['markIv'].values


        volfit_param = interpolate.splrep(strike, markIv, s=4)

        strikes_random = np.linspace(min(strike),max(strike),100)

        x = strikes_random
        y = interpolate.splev(strikes_random, volfit_param,ext=3)


        fig.add_trace(go.Scatter(x=x, y=y,
                      name='BTC '+str(e)[2:10],
                      line= dict(color=colore[i])),
                      row=1, col=1
                     )


        i = i + 1

    #GLOBAL CHARTS PARAMETERS
    fig.update_layout(showlegend=True)
    #fig.update_xaxes(range=[20000,70000],row=1,col=1)
    #base = max(50,min(y)-5)
    #fig.update_yaxes(range=[50,120])
    
    fig.update_layout(title=curr+" VOL CURVE STRIKE",title_x=0.50,
                      yaxis_title="volatility",
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=-0.25,
                          xanchor="center",
                          x=0.5
                                  ),
                   #['ggplot2', 'seaborn', 'simple_white', 'plotly',
                   #'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
                   #'ygridoff', 'gridon', 'none']
                  template = 'gridon')
    fig.show()



# COMPARISON BETWEEN EXCHANGES

In [8]:
exchange = ('deribit','bitcom','okex','delta')

df = pd.DataFrame()
data = pd.DataFrame()
for e in exchange:
    df = gvol_client.options_orderbook_details(exchange=e)
    df = pd.json_normalize(df['UtilityRealtimeOptionbook'])
    df['exchange'] = e
    data = data.append(df)
    
data['date'] = pd.to_datetime(data['date'],unit='ms')
data['expiration'] = pd.to_datetime(data['expiration'],unit='ms')
data['exp_date'] = data['expiration'].dt.date.astype(str)

data['DTE'] = pd.to_datetime(data['expiration'])- pd.to_datetime(data['date'],unit='ms')
data['DTE'] = data['DTE'].apply(datetime.timedelta.total_seconds) / (60*60*24)


data.loc[data['indexPrice'].isna(),'indexPrice'] = data.loc[data['indexPrice'].isna()]['underlyingPrice']

data['BSM'] = data.apply(BSM,axis=1)
data['delta_BSM'] = data.apply(lambda x: x['BSM'].callDelta if x['putCall']=='C' else x['BSM'].putDelta, axis=1)
data.loc[data['delta_BSM']>0,'delta_grid'] = 1 - data.loc[data['delta_BSM']>0]['delta_BSM']
data.loc[data['delta_BSM']<0,'delta_grid'] = -1 * data.loc[data['delta_BSM']<0]['delta_BSM']

#BUILD MONEYNESS
data.loc[(data['delta_grid']<0.50)&(data['putCall']=='P'),'moneyness'] = 'OTM'
data.loc[(data['delta_grid']>=0.50)&(data['putCall']=='P'),'moneyness'] = 'ITM'
data.loc[(data['delta_grid']<0.50)&(data['putCall']=='C'),'moneyness'] = 'ITM'
data.loc[(data['delta_grid']>=0.50)&(data['putCall']=='C'),'moneyness'] = 'OTM'

###FILTERING INSTRUMENTS FOR HAVING BETTER FITTING ON CURVE
data = data[(data['delta_grid'].between(0.02,0.97,inclusive=True))]

#data = data[data['moneyness']=='OTM']

In [9]:
data.exp_date.unique()

array(['2022-10-15', '2022-10-16', '2022-10-21', '2022-10-28',
       '2022-11-04', '2022-11-25', '2022-12-30', '2023-03-31',
       '2023-06-30', '2023-09-29', '2023-01-27'], dtype=object)

In [10]:
data.currency.unique()

array(['BTC', 'ETH', 'SOL', 'BNB', 'LINK', 'XRP'], dtype=object)

In [11]:
##SELECT SINGLE EXPIRY/CURRENCY FROM ABOVE ONES
exp_date = '2022-06-24'
currency = 'BTC'

data_exchange = data[(data['exp_date']==exp_date)&(data['currency']==currency)]


#DEFINE SUBPLOTS CHART
fig = go.Figure()

for exch in data_exchange['exchange'].unique():
    
    orderbookskew_exp = data_exchange[data_exchange['exchange']==exch].sort_values('strike')
    
    ##CHART FIT CURVE WITH ONLY LIQUID PART OF THE CHAIN
    orderbookskew_exp = orderbookskew_exp[orderbookskew_exp['moneyness']=='OTM']

    if len(orderbookskew_exp)<=3:
        continue

    strike = orderbookskew_exp['strike'].values
    markIv = orderbookskew_exp['markIv'].values


    volfit_param = interpolate.splrep(strike, markIv, s=4)

    strikes_random = np.linspace(min(strike),max(strike),100)

    x = strikes_random
    y = interpolate.splev(strikes_random, volfit_param,ext=3)


    fig.add_trace(go.Scatter(x=x, y=y,
                  name=exch.upper(),showlegend=True))

fig.update_layout(title='VOL FIT '+currency+" "+exp_date,title_x=0.50,
                  yaxis_title="volatility",
                  legend=dict(
                      orientation="h",
                      yanchor="bottom",
                      y=-0.25,
                      xanchor="center",
                      x=0.5
                              ),
               #['ggplot2', 'seaborn', 'simple_white', 'plotly',
               #'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
               #'ygridoff', 'gridon', 'none']
              template = 'gridon')
fig.show()


#DEFINE SUBPLOTS CHART
fig = make_subplots(specs=[[{"secondary_y": True}]])

for exch in data_exchange['exchange'].unique():
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    orderbookskew_exp = data_exchange[data_exchange['exchange']==exch].sort_values('strike')
    
    ##CHART OPEN INTEREST PROFILE
    ##Put
    x = orderbookskew_exp[orderbookskew_exp['putCall']=='P']['strike']
    y = orderbookskew_exp[orderbookskew_exp['putCall']=='P']['oi']
    fig.add_trace(go.Bar(x=x,y=y,name='put',marker_color='red',marker=dict(opacity=0.7)),secondary_y=True)
    ##Call
    x = orderbookskew_exp[orderbookskew_exp['putCall']=='C']['strike']
    y = orderbookskew_exp[orderbookskew_exp['putCall']=='C']['oi']
    fig.add_trace(go.Bar(x=x,y=y,name='call',marker_color='green',marker=dict(opacity=0.7)),secondary_y=True)

    ##CHART FIT CURVE WITH ONLY LIQUID PART OF THE CHAIN
    orderbookskew_exp = orderbookskew_exp[orderbookskew_exp['moneyness']=='OTM']
    
    if len(orderbookskew_exp)<=3:
        continue

    strike = orderbookskew_exp['strike'].values
    markIv = orderbookskew_exp['markIv'].values


    volfit_param = interpolate.splrep(strike, markIv, s=4)

    strikes_random = np.linspace(min(strike),max(strike),100)

    x = strikes_random
    y = interpolate.splev(strikes_random, volfit_param,ext=3)


    fig.add_trace(go.Scatter(x=x, y=y,name='Vol fit',marker_color='grey',showlegend=False))
    

    


    fig.update_yaxes(title_text='open interest', secondary_y=True)
    fig.update_layout(title=exch.upper()+" "+currency+" "+exp_date,title_x=0.50,
                      yaxis_title="volatility",
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=-0.25,
                          xanchor="center",
                          x=0.5
                                  ),
                   #['ggplot2', 'seaborn', 'simple_white', 'plotly',
                   #'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
                   #'ygridoff', 'gridon', 'none']
                  template = 'gridon')
    fig.show()